# Validation

- One way of validation is **required** parameter in schema. 
- In this example we add required parameter to *name* property. If we create new course without name and try to save it, we will get an exception.
- This exception basycally means that we are dealing with a promise, this promise is rejected but we have not handle that properly.

In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: String,
    author: String,
    tags: [ String ],
    date: { type: Date, default: Date.now},
    isPublished: Boolean
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Node.js Course',
        author: 'Mosh',
        tags: ['node', 'backend'],
        isPublished: true
    });

    const result = await course.save();
    console.log(result);
}

createCourse();


// console
// C:\Users\sebastian\Desktop\nodejs\08_Mongo - Data Validation>node index.js


// output
// C:\Users\sebastian\Desktop\nodejs\node_modules\mongoose\lib\document.js:3013
//     this.$__.validationError = new ValidationError(this);
//                                ^

// ValidationError: Course validation failed: name: Path `name` is required.

- To handle the rejected promise properly, we use **try-catch**.


In [ ]:
            .
            .
            .

async function createCourse(){
    const course = new Course({
        author: 'Mosh',
        tags: ['node', 'backend'],
        isPublished: true
    });
    
    try{
        const result = await course.save();
        console.log(result);
    }
    catch (ex){
        console.log(ex.message)
    }
}

            .
            .
            .

// console
// C:\Users\sebastian\Desktop\nodejs\08_Mongo - Data Validation>node index.js


// output
// Course validation failed: name: Path `name` is required.
// Connected to mongoDB...

- We can also manually trigger the validaton by using **validate** method of **course** object (**course.validate()**).
- The validation that we've implemented on the **name** property is only meaningfull in mongoose. MongoDB does not care about this name property. If we work with dabases like sql server or mysql, we can define validation at the database level. For example mark a column as required. In mongoDB we don't have that.

- Previously we used **Joy** for validation. Both Joy and mongoose validation complement each other.

# Built-in Validators

- **required** validator can be set to a simple boolean, or a function to condicionally make a property required. We cannot replace function with an arrow function, because it does not have it own **this**.

- Depending on the type of properties, we have aditional **buitl-in validators**. For example. For Strings we have **minlength**, **maxlength**, **match**, **enum**. For numbers we have **min** and **max

- In this example we are going to require the **price** property if **isPublished** is true; we add some built-in validators to name and price properties and we add new category with **enum** buit-in validator. 




In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: { 
        type: String,
        required: true,
        minlength: 5,
        maxlength: 255,
        // match: /pattern/ // Regexp 
    },
    category:{
        type: String,
        required: true,
        enum: ['web', 'mobile', 'network'] 
    },
    author: String,
    tags: [ String ],
    date: { type: Date, default: Date.now},
    isPublished: Boolean,
    price: {
        type: Number,
        required: function() {return this.isPublished;}, //function(){} can´t be replaced with () => {}
        min: 10,
        max: 20
    }
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Nodejs',
        category: 'web',
        author: 'New Author',
        tags: ['node', 'backend'],
        isPublished: true,
        price: 15
    });
    
    try{
        await course.validate()
        course.validate()
    }
    catch (ex){
        console.log(ex.message)
    }
}

createCourse();


# Custom Validators

- With **validate** property we can build a custom **vlidator**. 
- In this example we will create a custom validator for *tags* property. The objective is that the tags have at least one tag
- If we create a course with tags empty, null or without tags, the output after run index.js will show the message *A course should have at least one tag*

In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: { 
        type: String,
        required: true,
        minlength: 5,
        maxlength: 255,
        // match: /pattern/ // Regexp 
    },
    category:{
        type: String,
        required: true,
        enum: ['web', 'mobile', 'network'] 
    },
    author: String,
    tags: {
        type: Array,
        validate: {
            validator: function(v){
                return v && v.length > 0
            },
            message: 'A course should have at least one tag'
        }
    },
    date: { type: Date, default: Date.now},
    isPublished: Boolean,
    price: {
        type: Number,
        required: function() {return this.isPublished;}, //function(){} can´t be replaced with () => {}
        min: 10,
        max: 20
    }
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Nodejs',
        category: 'web',
        author: 'New Author',
        tags: ['x'],
        isPublished: true,
        price: 15
    });
    
    try{
        const result = await course.save();
        console.log(result);
    }
    catch (ex){
        console.log(ex.message)
    }
}

createCourse();


# Async validator

Sometimes validation logic may involve reading something from a database or from a remote http service therefore we don´have the answer straight away. In that case we need an asynchronous valitador.

- We simulate a process that takes some time with *setTimeout* function, and set **isAsync** parameter and retrun a **promise** in validator function.

- For example, we will try to create a new course with empty tags. The app will take 4 seconds to show error message *A course should have at least one tag*

In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: { 
        type: String,
        required: true,
        minlength: 5,
        maxlength: 255,
        // match: /pattern/ // Regexp 
    },
    category:{
        type: String,
        required: true,
        enum: ['web', 'mobile', 'network'] 
    },
    author: String,
    tags: {
        type: Array,
        validate: {
            isAsync: true,
            validator: function(v){
                return new Promise((resolve) => {
                    setTimeout(() => {
                        const result = v && v.length > 0;
                        resolve(result);
                    }, 4000);
                });
            },
            message: 'A course should have at least one tag'
        }
    },
    date: { type: Date, default: Date.now},
    isPublished: Boolean,
    price: {
        type: Number,
        required: function() {return this.isPublished;}, //function(){} can´t be replaced with () => {}
        min: 10,
        max: 20
    }
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Nodejs',
        category: 'web',
        author: 'New Author',
        tags: [],
        isPublished: true,
        price: 15
    });
    
    try{
        const result = await course.save();
        console.log(result);
    }
    catch (ex){
        console.log(ex.message)
    }
}

createCourse();


# Validation Errors

The exception that we get in **catch** block has a property called **errors** (**ex.errors**).

For example if we try to create a new course wit category "-" and tags "null", we will get two errors. The objective is to get these errors separately.

In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: { 
        type: String,
        required: true,
        minlength: 5,
        maxlength: 255,
        // match: /pattern/ // Regexp 
    },
    category:{
        type: String,
        required: true,
        enum: ['web', 'mobile', 'network'] 
    },
    author: String,
    tags: {
        type: Array,
        validate: {
            isAsync: true,
            validator: function(v){
                return new Promise((resolve) => {
                    setTimeout(() => {
                        const result = v && v.length > 0;
                        resolve(result);
                    }, 4000);
                });
            },
            message: 'A course should have at least one tag'
        }
    },
    date: { type: Date, default: Date.now},
    isPublished: Boolean,
    price: {
        type: Number,
        required: function() {return this.isPublished;}, //function(){} can´t be replaced with () => {}
        min: 10,
        max: 20
    }
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Nodejs',
        category: 'web',
        author: 'New Author',
        tags: [],
        isPublished: true,
        price: 15
    });
    
    try{
        const result = await course.save();
        console.log(result);
    }
    catch (ex){
        for (field in ex.errors) {}
        console.log(ex.message)
    }
}

createCourse();


// output
//------------------------------------
// Connected to mongoDB...
// `-` is not a valid enum value for path `category`.
// A course should have at least one tag


# SchemaType Options

There are a few more useful properties that we can add to attributes in schema:

For string:

**lowercase/uppercase**: mongoose automatically convert the value in lowercase/uppercase.For example, we add *lowercase* propertie to **category** attribute. if we create a course with category=WeB, this course will be stored with category=web.  

**trim**: mongoose remove the padings (spaces at the beginning or at the end) in input string.

For Number:

**get/set**: allow to apply an operation to input value using arrow function. For example, we add *get* and *set* properties to **price** attribute and implement a *round* operation in both. If we create a course with price = 15.8, the set function will be called, and the course will be stored with price=16. If we edit in mongoDB Compass the value of price to 16.8 and we read this course with *getCourses* function, the *get* propertie will be called and we will get a price=17.

In [ ]:
//index.js
//-------------------------------------------------------------
const mongoose = require('mongoose');

mongoose.connect('mongodb://localhost/playground')
    .then(() => console.log('Connected to mongoDB...'))
    .catch(err => console.error('Could not connect to mongoDB...'))

const courseSchema = new mongoose.Schema({
    name: { 
        type: String,
        required: true,
        minlength: 5,
        maxlength: 255,
        // match: /pattern/ // Regexp 
    },
    category:{
        type: String,
        required: true,
        enum: ['web', 'mobile', 'network'],
        lowercase: true,
        trim: true 
    },
    author: String,
    tags: {
        type: Array,
        validate: {
            isAsync: true,
            validator: function(v){
                return new Promise((resolve) => {
                    setTimeout(() => {
                        const result = v && v.length > 0;
                        resolve(result);
                    }, 4000);
                });
            },
            message: 'A course should have at least one tag'
        }
    },
    date: { type: Date, default: Date.now},
    isPublished: Boolean,
    price: {
        type: Number,
        required: function() {return this.isPublished;}, //function(){} can´t be replaced with () => {}
        min: 10,
        max: 20,
        set: v => Math.round(v),
        get: v => Math.round(v)
    }
});

const Course = mongoose.model('Course', courseSchema);

async function createCourse(){
    const course = new Course({
        name: 'Nodejs',
        category: ' WeB ',
        author: 'Sebastian',
        tags: ['backend'],
        isPublished: true,
        price: 15.8
    });
    
    try{
        const result = await course.save();
        console.log(result);
    }
    catch (ex){
        for (field in ex.errors) {
            // console.log(ex.errors[field])// for complete trace
            console.log(ex.errors[field].message)// just the message error
        }
    }
}

createCourse();

//output
//---------------------------------------------------
// Connected to mongoDB...
// {
//   name: 'Nodejs',
//   category: 'web',
//   author: 'Sebastian',
//   tags: [ 'backend' ],
//   isPublished: true,
//   price: 16,
//   _id: new ObjectId("62ec67093c942ec3878ae245"),
//   date: 2022-08-05T00:40:41.700Z,
//   __v: 0
// }


# Project

We create a new project called **Restaurant** which we add **dishes** and **customers** APIs based on **Courses** project.

# Restructuring the Project

- If we look at the Customer model, this is not a complex model. In real world application this model will be more complex. 
- *To keep the apps maintainable we shpuld ensure that each model is responsible for only one thing*. That is the single responsibility principle in practice.
- In the project, the customers module is part of the routes folder. Technically all we should have in this module is the definition of our customers routes. The definition of customer object does not belong to this module.
- We create new folder called **models** in wich we are going to have new customer.js and dish.js modules. Then, We move **Customer** model definition and **validationCustomer** function in routes/customers.js to models/customer.js and the same process with Dish. 
- In this way, **models/customer.js module** has the code for defining and validating customer object, and **routes/customers.js module** cointains the routes to work with customers (Same for dish.js and dishes.js modules).

In [ ]:
//models/customer.js
//----------------------------------------------------
const Joi = require('joi');
const mongoose = require('mongoose');

const Customer = mongoose.model('Customer', new mongoose.Schema({
    name: {
        type: String,
        minlength: 3,
        required: true
    },
    isGold: {
        type: Boolean,
        default: false,
    },
    phone: {
        type: String,
        required: true,
        minlength: 7
    }
}));

function validateCustomer(customer){
    const schema = Joi.object({
        name: Joi.string()
            .min(3)
            .required(),
        isGold: Joi.boolean(),
        phone: Joi.string()
            .min(7)
            .required()        
    })

    return schema.validate(customer);
}

module.exports.Customer = Customer;
module.exports.validate = validateCustomer;

// models/dish.js
//----------------------------------------------------
const Joi = require('joi');
const mongoose = require('mongoose');

const Dish = mongoose.model('Dish', new mongoose.Schema({
    name: {
        type: String,
        minlength: 3,
        required: true
    },
    price: {
        type: Number,
        required: true
    }
}));

function validateDish(dish){
    const schema = Joi.object({
        name: Joi.string()
            .min(3)
            .required(),
        price: Joi.number()
            .required()        
    })

    return schema.validate(dish);
}

module.exports.Dish = Dish;
module.exports.validate = validateDish;


In [ ]:
//routes/customers.js
//----------------------------------------------------
const { Customer, validate } = require('../models/customer');
const express = require('express');
const router = express.Router();


router.get('/', async (req, res) => {
    const customers = await Customer.find().sort('name');
    res.send(customers);
});

router.get('/:id', async (req, res) => {
    const customer = await Customer.findById(req.params.id);
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);
    
    let customer = new Customer(req.body);
    customer = await customer.save();
    res.send(customer);
});

router.put('/:id', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);

    const customer = await Customer.findByIdAndUpdate(req.params.id, req.body, {new: true}); 
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

router.delete('/:id', async (req, res) => {
    const customer = await Customer.findByIdAndDelete(req.params.id); 
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

module.exports = router;


//routes/dishes.js
//----------------------------------------------------
const {Dish, validate} = require('../models/dish')
const express = require('express');
const router = express.Router();


router.get('/', async (req, res) => {
    const dishes = await Dish.find().sort('name');
    res.send(dishes);
});

router.get('/:id', async (req, res) => {
    const dish = await Dish.findById(req.params.id);
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);
    
    let dish = new Dish(req.body);
    dish = await dish.save();
    res.send(dish);
});

router.put('/:id', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);

    const dish = await Dish.findByIdAndUpdate(req.params.id, req.body, {new: true}); 
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

router.delete('/:id', async (req, res) => {
    const dish = await Dish.findByIdAndDelete(req.params.id); 
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

module.exports = router;


In [ ]:
//index.js
//----------------------------------------------------
const mongoose = require('mongoose');
const express = require('express');
const app = express();
const dishes = require('./router/dishes');
const customers = require('./router/customers');

app.use(express.json());
app.use('/api/dishes', dishes);
app.use('/api/customers', customers);

mongoose.connect('mongodb://localhost/restaurant')
    .then(() => {console.log('Connected to mongoDB...')})
    .catch(err => {console.error('Could not connect to mongoDB...')});

const port = 3000 || process.env.PORT;
app.listen(port, () => {
    console.log( `Listening on port ${port}...` );
})
